In [386]:
import requests
import json
import os
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import folium
import openrouteservice as ors
import vroom
import geopy
from geopy.distance import great_circle

In [367]:
def get_token():
    load_dotenv('../.env')
    email = os.environ.get("email")
    password = os.environ.get("password")
    url = "https://openapi.emtmadrid.es/v3/mobilitylabs/user/login/"
    headers = {"email": email, "password" : password}
    response = requests.get(url, headers=headers)
    return response.content

In [368]:
def get_stations():
    load_dotenv('../.env')
    token = os.environ.get("access_token")
    url = "https://openapi.emtmadrid.es/v3/transport/bicimad/stations/"
    headers = {"accessToken" : token}
    response = requests.get(url, headers = headers).json()
    return response

In [369]:
stations = get_stations()
stations_real_time = pd.DataFrame(stations["data"])
stations_real_time[["longitude", "latitude"]] = stations_real_time["geometry"].apply(lambda x: pd.Series(x["coordinates"]))
stations_real_time = stations_real_time.drop(["geofence", "activate", "geometry", "integrator", "reservations_count", "no_available", "tipo_estacionPBSC", "virtualDelete", "virtual_bikes", "virtual_bikes_num", "code_suburb", "geofenced_capacity", "bikesGo"], axis=1)
stations_real_time

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
0,"Avenida del Ensanche de Vallecas, 9,",14,13,2190,2,"453 - Avenida del Ensanche de Vallecas, 9",453,27,18,-3.612530,40.370440
1,"Paseo de la Chopera,33,Comunidad de Madrid España",16,7,2205,2,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000
2,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100
3,"Paseo de la Castellana nº 164,Comunidad de Mad...",5,18,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137
4,"Guetaria , 84b,",19,5,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168
...,...,...,...,...,...,...,...,...,...,...,...
610,"Calle Francisco Balseiro nº 1,",0,0,1643,3,237 - Pablo Iglesias,237,0,06,-3.710525,40.451333
611,"237- calle Francisco Balseiro,Comunidad de Mad...",0,0,2362,3,1664 - 237- calle Francisco Balseiro,1664,0,06,-3.710274,40.451682
612,"Calle José Abascal frente al nº 2,",12,11,1607,2,201 - Canal,201,23,07,-3.703833,40.438694
613,"Calle Albelda, 6,",0,0,2316,3,"528 - Albelda, 6",528,24,15,-3.657367,40.449422


In [370]:
stations_high_realtime = stations_real_time[stations_real_time["light"] == 1].reset_index(drop=True)
stations_high_realtime

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
0,"Guetaria , 84b,",19,5,2226,1,395b - 395 - Guetaria 84b,395b,24,12,-3.715691,40.369168
1,"Calle de Gloria Fuertes, 181,",22,5,2151,1,"592 - Calle de Gloria Fuertes, 181",592,27,08,-3.724953,40.495697
2,"Calle del Pinzon, 1A,",18,5,2152,1,"376 - Calle del Pinzon, 1A",376,23,11,-3.740544,40.392292
3,"Av Abrantes, 55,",19,4,2153,1,"377 - Av Abrantes, 55",377,24,11,-3.727836,40.380918
4,"Calle Troya, 11,",21,3,2157,1,"519 - Calle Troya, 11",519,24,20,-3.611915,40.436996
...,...,...,...,...,...,...,...,...,...,...,...
104,"Manuel Noya , 29,",22,2,2033,1,"393 - Manuel Noya , 29",393,24,12,-3.711798,40.385379
105,"Calle Valdesangil 23,",23,1,2255,1,319 - Valdesangil 23,319,24,09,-3.716673,40.468422
106,"Calle del Sorbe , 20,",20,4,2070,1,"357 - Calle del Sorbe , 20",357,24,11,-3.713209,40.389870
107,"Avenida del Mediterráneo nº 19,",17,7,1478,1,76 - Puerta de Mariano de Cavia,76,24,03,-3.675102,40.407271


In [371]:
stations_low_realtime = stations_real_time[stations_real_time["light"] == 0].reset_index(drop=True)
stations_low_realtime

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
0,"Paseo de la Castellana nº 122,",2,21,2224,0,150 - Castellana frente a Hermanos Pinzón,150,23,05,-3.690800,40.449100
1,"Paseo de la Castellana nº 164,Comunidad de Mad...",5,18,2225,0,157- Castellana 164,157,23,05,-3.689415,40.459137
2,"Calle de Mota del Cuervo, 70,",2,22,2154,0,"539 - Calle de Mota del Cuervo, 70",539,24,16,-3.631357,40.464876
3,"Calle Seseña, 93,",1,23,2155,0,"334 - Seseña, 93",334,24,10,-3.768156,40.394973
4,"Calle Caleruega, 18,",1,23,2160,0,"495 - Calle Caleruega, 18",495,24,15,-3.670588,40.478746
...,...,...,...,...,...,...,...,...,...,...,...
286,"Calle norte nº 8,",1,18,1419,0,15 - Norte,15,19,01,-3.707445,40.426238
287,"Calle San Germán nº 57,",2,21,1561,0,155 - San Germán 57,155,23,06,-3.700930,40.457334
288,"467 - Calle Villardondiego frente, 32,",3,21,2261,0,467 - Calle Villardondiego 32,467,24,19,-3.608489,40.406236
289,"Calle Hernani, 59,",0,23,1995,0,"295 - Calle Hernani, 59",295,23,06,-3.697200,40.448100


In [372]:
def get_stations_real_time(df1, df2): #df1 = high, df2 = low
    data_list = []
    df1_lat_rad = np.radians(df1['latitude'].to_numpy())
    df1_lon_rad = np.radians(df1['longitude'].to_numpy())
    df2_lat_rad = np.radians(df2['latitude'].to_numpy())
    df2_lon_rad = np.radians(df2['longitude'].to_numpy())
    dlat = df2_lat_rad[:, np.newaxis] - df1_lat_rad
    dlon = df2_lon_rad[:, np.newaxis] - df1_lon_rad

    a = np.sin(dlat / 2) ** 2 + np.cos(df2_lat_rad[:, np.newaxis]) * np.cos(df1_lat_rad) * np.sin(dlon / 2) ** 2
    c = 2 * np.arcsin(np.sqrt(a))
    distance_matrix = c * 6371000 

    min_distance_indices = np.argmin(distance_matrix, axis=0)

    for x in range(len(df1["address"])):
        station_index = min_distance_indices[x]
        station_low_name = df2['name'].iloc[station_index]
        station_low_latitude = df2["latitude"].iloc[station_index]
        station_low_longitude = df2["longitude"].iloc[station_index]
        station_low_bikes = df2["dock_bikes"].iloc[station_index]
        station_low_code_district = df2["code_district"].iloc[station_index]
        station_high_name = df1["name"][x]
        station_high_latitude = df1["latitude"][x]
        station_high_longitude = df1["longitude"][x]
        station_high_bikes = df1["dock_bikes"][x]
        code_district  = df1["code_district"][x]
        min_distance = round(distance_matrix[station_index, x], 2)
        data_list.append({"code_district ":code_district , "station_high_name": station_high_name, "station_high_latitude": station_high_latitude, "station_high_longitude" : station_high_longitude, "station_high_bikes" : station_high_bikes, "station_low_code_district" : station_low_code_district, "station_low_name": station_low_name, "station_low_latitude": station_low_latitude, "station_low_longitude" : station_low_longitude, "station_low_bikes" : station_low_bikes,  "distance": min_distance})        
    stations_real_time = pd.DataFrame(data_list)
    return stations_real_time

In [373]:
stations_real_time2 = get_stations_real_time(stations_high_realtime, stations_low_realtime)
stations_real_time2

,station_high_code_district,station_high_name,station_high_latitude,station_high_longitude,station_high_bikes,station_low_code_district,station_low_name,station_low_latitude,station_low_longitude,station_low_bikes,distance
0,12,395b - 395 - Guetaria 84b,40.369168,-3.715691,19,17,"400 - Paseo de los Gigantes y Cabezudos, 4",40.360647,-3.693997,6,2067.87
1,08,"592 - Calle de Gloria Fuertes, 181",40.495697,-3.724953,22,08,"591 - Av.Ventisquero de la Condesa, 39",40.491082,-3.716229,5,898.59
2,11,"376 - Calle del Pinzon, 1A",40.392292,-3.740544,18,11,"366 - Paseo de Marcelino Camacho, 53B",40.387737,-3.744521,2,608.29
3,11,"377 - Av Abrantes, 55",40.380918,-3.727836,19,11,"365 - Av de la plaza de Toros , 3",40.385372,-3.737259,6,939.33
4,20,"519 - Calle Troya, 11",40.436996,-3.611915,21,20,"498 - Calle Alcala, 546",40.443133,-3.617832,6,846.44
...,...,...,...,...,...,...,...,...,...,...,...
104,12,"393 - Manuel Noya , 29",40.385379,-3.711798,22,11,212 - Marqués de Vadillo,40.398247,-3.716591,1,1487.35
105,09,319 - Valdesangil 23,40.468422,-3.716673,23,06,"303 - Paseo de la Dirección, 161",40.467789,-3.708085,1,729.87
106,11,"357 - Calle del Sorbe , 20",40.389870,-3.713209,20,11,212 - Marqués de Vadillo,40.398247,-3.716591,1,974.50
107,03,76 - Puerta de Mariano de Cavia,40.407271,-3.675102,17,03,82 - Niño Jesús,40.408456,-3.669753,4,471.69


In [374]:
stations_real_time["station_high_coordinates"] = stations_real_time[['station_high_longitude', 'station_high_latitude']].apply(lambda x: [x['station_high_longitude'], x['station_high_latitude']], axis=1)
stations_real_time = stations_real_time.drop(['station_high_longitude', 'station_high_latitude'], axis=1)
stations_real_time["station_low_coordinates"] = stations_real_time[['station_low_longitude', 'station_low_latitude']].apply(lambda x: [x['station_low_longitude'], x['station_low_latitude']], axis=1)
stations_real_time = stations_real_time.drop(['station_low_longitude', 'station_low_latitude'], axis=1)
stations_real_time

,station_high_code_district,station_high_name,station_high_bikes,station_low_code_district,station_low_name,station_low_bikes,distance,station_high_coordinates,station_low_coordinates
0,12,395b - 395 - Guetaria 84b,19,17,"400 - Paseo de los Gigantes y Cabezudos, 4",6,2067.87,"[-3.71569102, 40.36916772]","[-3.69399661, 40.36064659]"
1,08,"592 - Calle de Gloria Fuertes, 181",22,08,"591 - Av.Ventisquero de la Condesa, 39",5,898.59,"[-3.7249527, 40.4956969]","[-3.71622921, 40.49108225]"
2,11,"376 - Calle del Pinzon, 1A",18,11,"366 - Paseo de Marcelino Camacho, 53B",2,608.29,"[-3.7405438, 40.39229225]","[-3.744521, 40.387737]"
3,11,"377 - Av Abrantes, 55",19,11,"365 - Av de la plaza de Toros , 3",6,939.33,"[-3.72783615, 40.38091762]","[-3.73725908, 40.38537202]"
4,20,"519 - Calle Troya, 11",21,20,"498 - Calle Alcala, 546",6,846.44,"[-3.61191521, 40.43699597]","[-3.6178317, 40.44313349999999]"
...,...,...,...,...,...,...,...,...,...
104,12,"393 - Manuel Noya , 29",22,11,212 - Marqués de Vadillo,1,1487.35,"[-3.7117979, 40.3853787]","[-3.716591, 40.398247]"
105,09,319 - Valdesangil 23,23,06,"303 - Paseo de la Dirección, 161",1,729.87,"[-3.7166729, 40.4684218]","[-3.7080850848120273, 40.46778929738312]"
106,11,"357 - Calle del Sorbe , 20",20,11,212 - Marqués de Vadillo,1,974.50,"[-3.71320917, 40.3898701]","[-3.716591, 40.398247]"
107,03,76 - Puerta de Mariano de Cavia,17,03,82 - Niño Jesús,4,471.69,"[-3.675102, 40.407271]","[-3.6697526, 40.4084556]"


In [375]:
nearest_01 = stations_real_time[stations_real_time["code_district "]== "01"]
nearest_02 = stations_real_time[stations_real_time["code_district "]== "02"]
nearest_03 = stations_real_time[stations_real_time["code_district "]== "03"]
nearest_04 = stations_real_time[stations_real_time["code_district "]== "04"]
nearest_05 = stations_real_time[stations_real_time["code_district "]== "05"]
nearest_06 = stations_real_time[stations_real_time["code_district "]== "06"]
nearest_07 = stations_real_time[stations_real_time["code_district "]== "07"]
nearest_08 = stations_real_time[stations_real_time["code_district "]== "08"]
nearest_09 = stations_real_time[stations_real_time["code_district "]== "09"]
nearest_10 = stations_real_time[stations_real_time["code_district "]== "10"]
nearest_11 = stations_real_time[stations_real_time["code_district "]== "11"]
nearest_12 = stations_real_time[stations_real_time["code_district "]== "12"]
nearest_13 = stations_real_time[stations_real_time["code_district "]== "13"]
nearest_14 = stations_real_time[stations_real_time["code_district "]== "14"]
nearest_15 = stations_real_time[stations_real_time["code_district "]== "15"]
nearest_16 = stations_real_time[stations_real_time["code_district "]== "16"]
nearest_17 = stations_real_time[stations_real_time["code_district "]== "17"]
nearest_18 = stations_real_time[stations_real_time["code_district "]== "18"]
nearest_19 = stations_real_time[stations_real_time["code_district "]== "19"]
nearest_20 = stations_real_time[stations_real_time["code_district "]== "20"]
nearest_21 = stations_real_time[stations_real_time["code_district "]== "21"]


In [376]:
nearest_02

,station_high_code_district,station_high_name,station_high_bikes,station_low_code_district,station_low_name,station_low_bikes,distance,station_high_coordinates,station_low_coordinates
17,02,270 - Calle Teresa López Valcárcel,21,02,"274 - Méndez Alvaro, 73",6,1356.52,"[-3.69328771, 40.38862908]","[-3.68134346, 40.39675755]"
61,02,"276 - Paseo de las Yeserías, 15",20,02,"275 - Juan Antonio Vallejo Nájera Botas, 25",7,368.48,"[-3.7099166, 40.39900539]","[-3.7082123, 40.4020545]"
67,02,268 - Calle del Cobre (M. Planetario),17,02,"274 - Méndez Alvaro, 73",6,751.44,"[-3.689031463575172, 40.39338327980599]","[-3.68134346, 40.39675755]"
69,02,118 - Juan Martín,17,02,119 - Méndez Álvaro,2,176.61,"[-3.6882407, 40.400781]","[-3.6862653, 40.4012905]"
73,02,235 - Embajadores 191,23,02,"274 - Méndez Alvaro, 73",6,1189.79,"[-3.691076, 40.389041]","[-3.68134346, 40.39675755]"
74,02,236 - Paseo Imperial,22,01,"263 - Avda Gran Vía de San Francisco , 9",2,372.52,"[-3.7170785, 40.4075794]","[-3.712793282384579, 40.40833880815136]"
87,02,162 - Metro Pirámides,20,02,"275 - Juan Antonio Vallejo Nájera Botas, 25",7,256.51,"[-3.7105765, 40.4034968]","[-3.7082123, 40.4020545]"
88,02,164 - Paseo de las Delicias,24,02,168 - Batalla del Salado,3,826.75,"[-3.69461, 40.39723]","[-3.6959, 40.4046]"
89,02,167 - Segovia 45,24,01,166 - Segovia 26,7,348.58,"[-3.71748917032373, 40.41374770742206]","[-3.7133770233135133, 40.41390473235381]"
90,02,169 - Pirámides,18,02,280 - PASEO DE LOS MELANCÓLICOS 73,4,402.45,"[-3.713823, 40.401589]","[-3.718569, 40.401782]"


In [377]:
def get_coordinates(df):
    coordinates_list = []
    for index, row in df.iterrows():
        coordinates_list.append(row['station_high_coordinates'])
        coordinates_list.append(row['station_low_coordinates'])
    return coordinates_list


In [378]:
get_coordinates(nearest_02)

[[-3.69328771, 40.38862908],
 [-3.68134346, 40.39675755],
 [-3.7099166, 40.39900539],
 [-3.7082123, 40.4020545],
 [-3.689031463575172, 40.39338327980599],
 [-3.68134346, 40.39675755],
 [-3.6882407, 40.400781],
 [-3.6862653, 40.4012905],
 [-3.691076, 40.389041],
 [-3.68134346, 40.39675755],
 [-3.7170785, 40.4075794],
 [-3.712793282384579, 40.40833880815136],
 [-3.7105765, 40.4034968],
 [-3.7082123, 40.4020545],
 [-3.69461, 40.39723],
 [-3.6959, 40.4046],
 [-3.71748917032373, 40.41374770742206],
 [-3.7133770233135133, 40.41390473235381],
 [-3.713823, 40.401589],
 [-3.718569, 40.401782],
 [-3.6934926, 40.4008279],
 [-3.6959, 40.4046],
 [-3.6941876374343896, 40.391447085678806],
 [-3.68134346, 40.39675755],
 [-3.67589, 40.3940423],
 [-3.68134346, 40.39675755],
 [-3.67970176633988, 40.393023802369754],
 [-3.68134346, 40.39675755]]

In [379]:
load_dotenv('../.env')

headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'Authorization': os.environ.get("openroute_api_key"),
    'Content-Type': 'application/json; charset=utf-8'
}

vehicle_start = [-3.6823731969472644, 40.46209827032537]

coordinates_list = get_coordinates(nearest_02)

m = folium.Map(location = coordinates_list[0])

vehicles = []

for idx, coords in enumerate(coordinates_list):
    vehicles.append(ors.optimization.Vehicle(id=idx, profile = "driving-car", start = coords, end = coords, capacity = [50]))
    folium.Marker(location=list(reversed(coords)), icon=folium.Icon(icon="Van")).add_to(m)

shipments = []

for idx, coords in enumerate(coordinates_list):
    pickup = dict(id = idx, location= coords, description= "load")
    delivery = dict(id=idx, location = coords, description= "unload")
    shipments.append(ors.optimization.Shipment(pickup=pickup, delivery = delivery))

    folium.Marker(location = list(reversed(coords)), icon=folium.Icon(icon = "Work Route", color = "green")).add_to(m)
    
body = {"vehicles", "shipments"}

call = 

optimized = client.optimization(
    shipments = shipments,
    vehicles = vehicles,
    geometry = True,
)

m

SyntaxError: invalid syntax (3377566117.py, line 32)

VOY A ASUMIR QUE LOS FURGONES DE BICIMAD SALEN DE LA SEDE CENTRAL DE LA EMT EN MADRID (CALLE DEL CERRO DE LA PLATA, 4, MADRID)

In [380]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

# Calculo la ruta azul desde vehicle_start hasta la primera station_high_coordinate
route_start_to_first_high = client.directions(
    coordinates=[vehicle_start, nearest_01.iloc[0]['station_high_coordinates']],
    profile='driving-car',
    format='geojson'
)

# Calculo la ruta desde la última station_low_coordinate hasta vehicle_start
route_last_low_to_end = client.directions(
    coordinates=[nearest_01.iloc[-1]['station_low_coordinates'], vehicle_start],
    profile='driving-car',
    format='geojson'
)

# Creo un mapa con Folium
m = folium.Map(location=[40.4167, -3.70325], zoom_start=13)

# Añado marcador azul para vehicle start
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='Vehicle Start', icon=folium.Icon(color='blue')).add_to(m)

# Añado la ruta azul al mapa
folium.PolyLine(locations=[coord[::-1] for coord in route_start_to_first_high['features'][0]['geometry']['coordinates']],
                color='blue').add_to(m)

last_coord_blue = route_start_to_first_high['features'][0]['geometry']['coordinates'][-1]
folium.Marker(location=[last_coord_blue[1], last_coord_blue[0]], popup='Inicio de la ruta', icon=folium.Icon(color='blue')).add_to(m)

# Itero sobre cada fila del DataFrame y añado las rutas al mapa
for i in range(len(nearest_01)-1):
    # Coordenadas de carga y descarga
    coordinate_high_start = nearest_01.iloc[i]['station_high_coordinates']
    coordinate_low = nearest_01.iloc[i]['station_low_coordinates']
    coordinate_high_end = nearest_01.iloc[i + 1]['station_high_coordinates']
    
    # Calculo rutas 
    route_high_start = client.directions(coordinates=[vehicle_start, coordinate_high_start], profile='driving-car', format='geojson')
    route_low = client.directions(coordinates=[coordinate_high_start, coordinate_low], profile='driving-car', format='geojson')
    route_high_end = client.directions(coordinates=[coordinate_low, coordinate_high_end], profile='driving-car', format='geojson')
    
    # Añado la ruta verde al mapa
    folium.PolyLine(locations=[coord[::-1] for coord in route_low['features'][0]['geometry']['coordinates']],
                    color='green').add_to(m)
    
    # Añado marcador naranja para station_high_coordinates
    folium.Marker(location=[coordinate_high_start[1], coordinate_high_start[0]],
                  popup=nearest_01.iloc[i]['station_high_name'], icon=folium.Icon(color='orange')).add_to(m)
    
    # Añado marcador verde para station_low_coordinates
    folium.Marker(location=[coordinate_low[1], coordinate_low[0]],
                  popup=nearest_01.iloc[i]['station_low_name'], icon=folium.Icon(color='green')).add_to(m)

    # Calculo la ruta entre la estación baja actual y la siguiente estación alta
    route_low_to_high = client.directions(coordinates=[coordinate_low, coordinate_high_end], profile='driving-car', format='geojson')

    # Añado la ruta verde al mapa
    folium.PolyLine(locations=[coord[::-1] for coord in route_low_to_high['features'][0]['geometry']['coordinates']],
                    color='green').add_to(m)

    # Añado marcador verde donde acaba la línea verde
    folium.Marker(location=[route_low_to_high['features'][0]['geometry']['coordinates'][-1][1],
                            route_low_to_high['features'][0]['geometry']['coordinates'][-1][0]],
                  popup=nearest_01.iloc[i]['station_low_name'], icon=folium.Icon(color='orange')).add_to(m)

# Calculo la ruta desde la última station_low_coordinate hasta vehicle_start
route_last_low_to_end = client.directions(coordinates=[nearest_01.iloc[-1]['station_low_coordinates'], vehicle_start], profile='driving-car', format='geojson')

# Añado la ruta roja al mapa
folium.PolyLine(locations=[coord[::-1] for coord in route_last_low_to_end['features'][0]['geometry']['coordinates']],
                color='red').add_to(m)

# Añado un marcador verde donde comienza la línea roja
folium.Marker(location=[route_last_low_to_end['features'][0]['geometry']['coordinates'][0][1],
                        route_last_low_to_end['features'][0]['geometry']['coordinates'][0][0]],
              popup="Final de la ruta", icon=folium.Icon(color='red')).add_to(m)

m 

In [392]:
district_01 = stations_real_time[stations_real_time["code_district"]== "01"]
district_02 = stations_real_time[stations_real_time["code_district"]== "02"]
district_03 = stations_real_time[stations_real_time["code_district"]== "03"]
district_04 = stations_real_time[stations_real_time["code_district"]== "04"]
district_05 = stations_real_time[stations_real_time["code_district"]== "05"]
district_06 = stations_real_time[stations_real_time["code_district"]== "06"]
district_07 = stations_real_time[stations_real_time["code_district"]== "07"]
district_08 = stations_real_time[stations_real_time["code_district"]== "08"]
district_09 = stations_real_time[stations_real_time["code_district"]== "09"]
district_10 = stations_real_time[stations_real_time["code_district"]== "10"]
district_11 = stations_real_time[stations_real_time["code_district"]== "11"]
district_12 = stations_real_time[stations_real_time["code_district"]== "12"]
district_13 = stations_real_time[stations_real_time["code_district"]== "13"]
district_14 = stations_real_time[stations_real_time["code_district"]== "14"]
district_15 = stations_real_time[stations_real_time["code_district"]== "15"]
district_16 = stations_real_time[stations_real_time["code_district"]== "16"]
district_17 = stations_real_time[stations_real_time["code_district"]== "17"]
district_18 = stations_real_time[stations_real_time["code_district"]== "18"]
district_19 = stations_real_time[stations_real_time["code_district"]== "19"]
district_20 = stations_real_time[stations_real_time["code_district"]== "20"]
district_21 = stations_real_time[stations_real_time["code_district"]== "21"]

In [393]:
district_02

,address,dock_bikes,free_bases,id,light,name,number,total_bases,code_district,longitude,latitude
1,"Paseo de la Chopera,33,Comunidad de Madrid España",16,7,2205,2,"267 - Paseo de la Chopera, 33",267,23,02,-3.700400,40.395000
35,"Paseo Virgen del Puerto, 25,",0,0,2181,3,"BiciMAD5 - Paseo Virgen del Puerto, 25",BiciMAD5,0,02,-3.721009,40.411611
65,"Calle Ribera de Curtidores nº 28,",8,19,1450,0,46 - Ribera de Curtidores,46,27,02,-3.707126,40.405315
114,Calle Teresa López Valcárcel esq. Teniente Cor...,21,3,2071,1,270 - Calle Teresa López Valcárcel,270,24,02,-3.693288,40.388629
157,"Calle Delicias, 45,",8,11,2117,2,"273 - Calle Delicias, 45",273,19,02,-3.690918,40.403819
210,"Glorieta de Embajadores nº 6,",9,15,1451,2,47 - Embajadores 1,47,24,02,-3.702800,40.404577
234,"PASEO DE LOS MELANCÓLICOS 73,",4,19,2247,0,280 - PASEO DE LOS MELANCÓLICOS 73,280,23,02,-3.718569,40.401782
248,"271 - Calle Amaltea , 1,",14,10,2265,2,"271 - Calle Amaltea , 1",271,24,02,-3.683170,40.394706
250,"Paseo de Juan Antonio Vallejo Nájera Botas, 28,",7,16,2267,0,"275 - Juan Antonio Vallejo Nájera Botas, 25",275,23,02,-3.708212,40.402054
252,"Calle Méndez Alvaro, 73,",6,17,2269,0,"274 - Méndez Alvaro, 73",274,23,02,-3.681343,40.396758


In [389]:
def find_nearest_to_central(df, coords):
    station_coordinates = df['station_high_coordinates'].tolist()
    nearest_station = min(station_coordinates, key=lambda coord: great_circle(coord, vehicle_start).meters)
    return nearest_station

In [ ]:
load_dotenv('../.env')
client = ors.Client(key=os.environ.get("openroute_api_key"))

# Coordenadas de inicio (supongo que empiezan la ruta en la central de la EMT)
vehicle_start = [-3.6823731969472644, 40.46209827032537]

#Calcular la estación high más cercana a la central (esa va a ser la primera).

first_stop = find_nearest_to_central(nearest_02, vehicle_start)


# Calculo la ruta azul desde vehicle_start hasta la primera estación con un alto número de bicis. 
route_central_to_first_point = client.directions(
    coordinates=[vehicle_start, first_stop],
    profile='driving-car',
    format='geojson'
)

# Creo un mapa con Folium
m = folium.Map(location=vehicle_start, zoom_start=13)

# Añado marcador morado para la central (vehicle_start)
folium.Marker(location=[vehicle_start[1], vehicle_start[0]], popup='CENTRAL EMT', icon=folium.Icon(color='purple')).add_to(m)

# Añado la ruta azul al mapa
folium.PolyLine(locations=[coord[::-1] for coord in route_central_to_first_point['features'][0]['geometry']['coordinates']],
                color='blue').add_to(m)

marker_route_start = route_central_to_first_point['features'][0]['geometry']['coordinates'][-1]
folium.Marker(location=[marker_route_start[1], marker_route_start[0]], popup='Inicio de la ruta', icon=folium.Icon(color='blue')).add_to(m)

# Itero sobre cada fila del DataFrame y añado las rutas al mapa
for i in range(len(nearest_01)-1):
    # Coordenadas de carga y descarga
    coordinate_high_start = nearest_01.iloc[i]['station_high_coordinates']
    coordinate_low = nearest_01.iloc[i]['station_low_coordinates']
    coordinate_high_end = nearest_01.iloc[i + 1]['station_high_coordinates']
    
    # Calculo rutas 
    route_high_start = client.directions(coordinates=[vehicle_start, coordinate_high_start], profile='driving-car', format='geojson')
    route_low = client.directions(coordinates=[coordinate_high_start, coordinate_low], profile='driving-car', format='geojson')
    route_high_end = client.directions(coordinates=[coordinate_low, coordinate_high_end], profile='driving-car', format='geojson')
    
    # Añado la ruta verde al mapa
    folium.PolyLine(locations=[coord[::-1] for coord in route_low['features'][0]['geometry']['coordinates']],
                    color='green').add_to(m)
    
    # Añado marcador naranja para station_high_coordinates
    folium.Marker(location=[coordinate_high_start[1], coordinate_high_start[0]],
                  popup=nearest_01.iloc[i]['station_high_name'], icon=folium.Icon(color='orange')).add_to(m)
    
    # Añado marcador verde para station_low_coordinates
    folium.Marker(location=[coordinate_low[1], coordinate_low[0]],
                  popup=nearest_01.iloc[i]['station_low_name'], icon=folium.Icon(color='green')).add_to(m)

    # Calculo la ruta entre la estación baja actual y la siguiente estación alta
    route_low_to_high = client.directions(coordinates=[coordinate_low, coordinate_high_end], profile='driving-car', format='geojson')

    # Añado la ruta verde al mapa
    folium.PolyLine(locations=[coord[::-1] for coord in route_low_to_high['features'][0]['geometry']['coordinates']],
                    color='green').add_to(m)

    # Añado marcador verde donde acaba la línea verde
    folium.Marker(location=[route_low_to_high['features'][0]['geometry']['coordinates'][-1][1],
                            route_low_to_high['features'][0]['geometry']['coordinates'][-1][0]],
                  popup=nearest_01.iloc[i]['station_low_name'], icon=folium.Icon(color='orange')).add_to(m)

# Calculo la ruta desde la última station_low_coordinate hasta vehicle_start
route_last_low_to_end = client.directions(coordinates=[nearest_01.iloc[-1]['station_low_coordinates'], vehicle_start], profile='driving-car', format='geojson')

# Añado la ruta roja al mapa
folium.PolyLine(locations=[coord[::-1] for coord in route_last_low_to_end['features'][0]['geometry']['coordinates']],
                color='red').add_to(m)

# Añado un marcador verde donde comienza la línea roja
folium.Marker(location=[route_last_low_to_end['features'][0]['geometry']['coordinates'][0][1],
                        route_last_low_to_end['features'][0]['geometry']['coordinates'][0][0]],
              popup="Final de la ruta", icon=folium.Icon(color='red')).add_to(m)

m 